<a href="https://colab.research.google.com/github/graveo-wicaksana/DA_restaurantSales/blob/main/DA_restaurantSales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview
Explain the background

# Key Performance Indicators
1.   Finding most ordered item for each categories
2.   Listing the prefered items for certain payment method
3.   Explain the trend sales during a year to find high sales on certain months
4.   Recommend marketing strategic to increase sales and/or engage more customers



# Preparation Datasets
The dataset is obtained from kaggle by Ahmed Mohamed with title "Restaurant Sales-Dirty Data for Cleaning Training". I have downloaded the file and stored in google drive to prevent error execution in the future in case the author move the path of the file. Hopefully, people will download the file from kaggle's author in this [link](https://www.kaggle.com/datasets/ahmedmohamed2003/restaurant-sales-dirty-data-for-cleaning-training).

The dataset will be processed using Python in this Jupyter Notebook.

In [20]:
#Load Library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
#Load file and show top 5 records.
url = 'http://drive.google.com/file/d/1H1ya5-Dv4Pq-ony2SbsFMKvzZ3Yp_Df0/view?usp=sharing'
url = 'http://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
#Working Laptop for learning only. Use below if needed. Regular case use top of this code.
# url1 = 'C:/Users/GW37696X/Downloads/restaurant_sales_data.csv'
# df = pd.read_csv(url1)
df.head()

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
0,ORD_705844,CUST_092,Side Dishes,Side Salad,3.0,1.0,3.0,2023-12-21,Credit Card
1,ORD_338528,CUST_021,Side Dishes,Mashed Potatoes,4.0,3.0,12.0,2023-05-19,Digital Wallet
2,ORD_443849,CUST_029,Main Dishes,Grilled Chicken,15.0,4.0,60.0,2023-09-27,Credit Card
3,ORD_630508,CUST_075,Drinks,NaN,NaN,2.0,5.0,2022-08-09,Credit Card
4,ORD_648269,CUST_031,Main Dishes,Pasta Alfredo,12.0,4.0,48.0,2022-05-15,Cash


In [22]:
#Show info of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17534 entries, 0 to 17533
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order ID        17534 non-null  object 
 1   Customer ID     17534 non-null  object 
 2   Category        17534 non-null  object 
 3   Item            15776 non-null  object 
 4   Price           16658 non-null  float64
 5   Quantity        17104 non-null  float64
 6   Order Total     17104 non-null  float64
 7   Order Date      17534 non-null  object 
 8   Payment Method  16452 non-null  object 
dtypes: float64(3), object(6)
memory usage: 1.2+ MB


In [23]:
#Show descriptif statistic of numeric data type using describe
df.describe()

,Price,Quantity,Order Total
count,16658.000000,17104.000000,17104.000000
mean,6.586325,3.014149,19.914494
std,4.834652,1.414598,18.732549
min,1.000000,1.000000,1.000000
25%,3.000000,2.000000,7.500000
50%,5.000000,3.000000,15.000000
75%,7.000000,4.000000,25.000000
max,20.000000,5.000000,100.000000


In [24]:
#Show descriptif statistic of object data type using describe
df.describe(include="object")

,Order ID,Customer ID,Category,Item,Order Date,Payment Method
count,17534,17534,17534,15776,17534,16452
unique,17534,100,5,26,730,3
top,ORD_680707,CUST_066,Main Dishes,Pasta Alfredo,2023-11-25,Credit Card
freq,1,207,3551,998,42,5504


# Preprocessing Datasets

Do cleaning process to get clean dataset by checking and handling:
1. Duplicated records
2. Null records
3. Inconsistent records (i.e. format, and categories)
4. Typo records
5. Invalid records


## Handling Duplicated Records

In [25]:
#check duplicated records
df.duplicated().sum()

np.int64(0)

## Handling Null Value

In [26]:
#check null records.
print (df.isna().sum())
#total entries: 17534 entries
#weakenss: Not show directly the precentage

#try to show percentage
na_columns = [i for i in df.columns if df[i].isna().mean() > 0]
df_na_columns = df[na_columns].isna().mean().round(2)
print (df_na_columns)
#better use this to know the percentage

Order ID             0
Customer ID          0
Category             0
Item              1758
Price              876
Quantity           430
Order Total        430
Order Date           0
Payment Method    1082
dtype: int64
Item              0.10
Price             0.05
Quantity          0.02
Order Total       0.02
Payment Method    0.06
dtype: float64


Percentage of NaN values in 'Item' columns is about 10%. The NaN values will be imputed to prevent significant loss.

For the numerical value, it can be filled using logical calculation i.e Price * Quantity = Order Total

For payment method, it will be using modus value. 

### Item column

In [27]:
#To handling item values, use menu map from source. The menu map will be written in a new dataset
df_menumap = pd.DataFrame({
    'Category': ['Starters', 'Starters', 'Starters', 'Starters', 'Starters', 'Starters',
                'Main Dishes', 'Main Dishes', 'Main Dishes', 'Main Dishes', 'Main Dishes',
                'Desserts', 'Desserts', 'Desserts', 'Desserts', 'Desserts',
                'Drinks', 'Drinks', 'Drinks', 'Drinks', 'Drinks',
                'Side Dishes', 'Side Dishes', 'Side Dishes', 'Side Dishes', 'Side Dishes'],
    'Item' : ['Chicken Melt', 'French Fries', 'Cheese Fries', 'Sweet Potato Fries', 'Beef Chili', 'Nachos Grande',
             'Grilled Chicken', 'Steak', 'Pasta Alfredo', 'Salmon', 'Vegetarian Platter',
             'Chocolate Cake', 'Ice Cream', 'Fruit Salad', 'Cheesecake', 'Brownie',
             'Coca Cola', 'Orange Juice', 'Lemonade', 'Iced Tea', 'Water',
             'Mashed Potatoes', 'Grilled Vegetables', 'Side Salad', 'Garlic Bread', 'Onion Rings'],
    'Price' : [8.0, 4.0, 5.0, 5.0, 7.0, 10.0,
               15.0, 20.0, 12.0, 18.0, 14.0,
               6.0, 5.0, 4.0, 7.0, 6.0,
               2.5, 3.0, 3.0, 2.5, 1.0,
               4.0, 5.0, 3.0, 4.0, 5.0]
})

#preview menumap
df_menumap.sort_values(by=['Category', 'Price'], ascending=[True, True])

,Category,Item,Price
13,Desserts,Fruit Salad,4.0
12,Desserts,Ice Cream,5.0
11,Desserts,Chocolate Cake,6.0
15,Desserts,Brownie,6.0
14,Desserts,Cheesecake,7.0
20,Drinks,Water,1.0
16,Drinks,Coca Cola,2.5
19,Drinks,Iced Tea,2.5
17,Drinks,Orange Juice,3.0
18,Drinks,Lemonade,3.0


In [36]:
#keep the origin
df2 = pd.DataFrame(df) 

#keep the rows without NaN Values
df2_no_NaN = df2.dropna()

#Examine existing df with Na Values for each price
sum_nan_grouped = df2.groupby(['Category', 'Price'], dropna=False)['Item'].apply(lambda x: x.isna().sum())
print (sum_nan_grouped)

Category     Price
Desserts     4.0       30
             5.0       62
             6.0       70
             7.0       17
             NaN      180
Drinks       1.0       66
             2.5       60
             3.0       69
             NaN      183
Main Dishes  12.0      61
             14.0      21
             15.0      44
             18.0      23
             20.0      23
             NaN      181
Side Dishes  3.0       63
             4.0       50
             5.0       72
             NaN      160
Starters     4.0       49
             5.0       43
             7.0       26
             8.0       24
             10.0       9
             NaN      172
Name: Item, dtype: int64


In [29]:
#Examine menu with duplicate price for different items
duplicate_menu_cost = df_menumap.duplicated(subset=['Category', 'Price'], keep=False)
df_menumap[duplicate_menu_cost].sort_values(by=['Category', 'Price'], ascending=[True,False])
#Starters: 5| Side Dishes: 5 or 4| Drinks: 2.5 or 3| Desserts: 6
#Purpose: Know price with duplicate items, so it must be aware to handling NaN value with category and price like below

# dessert_6 = 'Chocolate Cake'
# drink_3 = 'Orange Juice'
# drink_2_5 = 'Coca Cola'
# sideD_5 = 'Grilled Vegetables'
# sideD_4 = 'Mashed Potatoes'
# starters_5 = 'Cheeese Fries'




,Category,Item,Price
11,Desserts,Chocolate Cake,6.0
15,Desserts,Brownie,6.0
17,Drinks,Orange Juice,3.0
18,Drinks,Lemonade,3.0
16,Drinks,Coca Cola,2.5
19,Drinks,Iced Tea,2.5
22,Side Dishes,Grilled Vegetables,5.0
25,Side Dishes,Onion Rings,5.0
21,Side Dishes,Mashed Potatoes,4.0
24,Side Dishes,Garlic Bread,4.0


In [30]:
#Variable Filter NaN Item
na_item = df2['Item'].isna()

#Handling Dessert Category for unique amount and non NaN
#Variable Filter Desserts
na_desserts = df2['Category'] == 'Desserts'
#Variable Filter Amount
na_desserts_unique_price = df2['Price'] != 6.0
#Execution
df2_na_desserts_unique_price = df2[na_desserts & na_desserts_unique_price & na_item]
merged_df2_na_desserts_unique_price = pd.merge(df2_na_desserts_unique_price, df_menumap, on=['Category', 'Price'], how='left')
merged_df2_na_desserts_unique_price.dropna(subset=['Price'], inplace=True)
merged_df2_na_desserts_unique_price['Item_x'] = merged_df2_na_desserts_unique_price['Item_x'].fillna(merged_df2_na_desserts_unique_price['Item_y'])
merged_df2_na_desserts_unique_price = merged_df2_na_desserts_unique_price.rename(columns={'Item_x': 'Item'})
merged_df2_na_desserts_unique_price.drop(columns=['Item_y'], inplace=True)

#Handling Drinks Category for unique amount and non NaN
#Variable Filter Drinks
na_drinks = df2['Category'] == 'Drinks'
#Variable Filter Amount
na_drinks_unique_price = df2['Price'] == 1.0
#Execution
df2_na_drinks_unique_price = df2[na_drinks & na_drinks_unique_price & na_item]
merged_df2_na_drinks_unique_price = pd.merge(df2_na_drinks_unique_price, df_menumap, on=['Category', 'Price'], how='left')
merged_df2_na_drinks_unique_price.dropna(subset=['Price'], inplace=True)
merged_df2_na_drinks_unique_price['Item_x'] = merged_df2_na_drinks_unique_price['Item_x'].fillna(merged_df2_na_drinks_unique_price['Item_y'])
merged_df2_na_drinks_unique_price = merged_df2_na_drinks_unique_price.rename(columns={'Item_x': 'Item'})
merged_df2_na_drinks_unique_price.drop(columns=['Item_y'], inplace=True)

#Handling Main Dishes Category for non NaN. All amounts are unique
#Variable Filter Main Dishes
na_main_dishes = df2['Category'] == 'Main Dishes'
#Execution
df2_na_main_dishes_unique_price = df2[na_main_dishes & na_item]
merged_df2_na_main_dishes_unique_price = pd.merge(df2_na_main_dishes_unique_price, df_menumap, on=['Category', 'Price'], how='left')
merged_df2_na_main_dishes_unique_price.dropna(subset=['Price'], inplace=True)
merged_df2_na_main_dishes_unique_price['Item_x'] = merged_df2_na_main_dishes_unique_price['Item_x'].fillna(merged_df2_na_main_dishes_unique_price['Item_y'])
merged_df2_na_main_dishes_unique_price = merged_df2_na_main_dishes_unique_price.rename(columns={'Item_x': 'Item'})
merged_df2_na_main_dishes_unique_price.drop(columns=['Item_y'], inplace=True)

#Handling Side Dishes Category for unique amount and non NaN
#Variable Filter Side Dishes
na_side_dishes = df2['Category'] == 'Side Dishes'
#Variable Filter Amount
na_side_dishes_unique_price = df2['Price'] == 3.0
#Execution
df2_na_side_dishes_unique_price = df2[na_side_dishes & na_side_dishes_unique_price & na_item]
merged_df2_na_side_dishes_unique_price = pd.merge(df2_na_side_dishes_unique_price, df_menumap, on=['Category', 'Price'], how='left')
merged_df2_na_side_dishes_unique_price.dropna(subset=['Price'], inplace=True)
merged_df2_na_side_dishes_unique_price['Item_x'] = merged_df2_na_side_dishes_unique_price['Item_x'].fillna(merged_df2_na_side_dishes_unique_price['Item_y'])
merged_df2_na_side_dishes_unique_price = merged_df2_na_side_dishes_unique_price.rename(columns={'Item_x': 'Item'})
merged_df2_na_side_dishes_unique_price.drop(columns=['Item_y'], inplace=True)

#Handling Starters Category for unique amount and non NaN
#Variable Filter Starters
na_starters = df2['Category'] == 'Starters'
#Variable Filter Amount
na_starters_unique_price = df2['Price'] != 5.0
#Execution
df2_na_starters_unique_price = df2[na_starters & na_starters_unique_price & na_item]
merged_df2_na_starters_unique_price = pd.merge(df2_na_starters_unique_price, df_menumap, on=['Category', 'Price'], how='left')
merged_df2_na_starters_unique_price.dropna(subset=['Price'], inplace=True)
merged_df2_na_starters_unique_price['Item_x'] = merged_df2_na_starters_unique_price['Item_x'].fillna(merged_df2_na_starters_unique_price['Item_y'])
merged_df2_na_starters_unique_price = merged_df2_na_starters_unique_price.rename(columns={'Item_x': 'Item'})
merged_df2_na_starters_unique_price.drop(columns=['Item_y'], inplace=True)


#Check na
# df2[na_desserts].groupby(['Category', 'Price'], dropna=False)['Item'].apply(lambda x: x.isna().sum())

In [39]:
#DUMMY_CHECK
# print (sum_nan_grouped)
print (df2_no_NaN.shape[0])
print (merged_df2_na_desserts_unique_price.shape[0])
print (merged_df2_na_drinks_unique_price.shape[0])
print (merged_df2_na_main_dishes_unique_price.shape[0])
print (merged_df2_na_side_dishes_unique_price.shape[0])
print (merged_df2_na_starters_unique_price.shape[0])

#Start to handle na items for same price in the same category

15252
109
66
172
63
108


In [82]:
#check modus number for non unique price of NaN items for each category
#desserts
nan_desserts_p6 = (df2_no_NaN['Category'] == 'Desserts') & (df2_no_NaN['Price'] == 6.0)
desserts_p6 = df2_no_NaN[nan_desserts_p6][['Category', 'Item']]['Item'].mode([0])

# drinks
nan_drinks_p3 = (df2_no_NaN['Category'] == 'Drinks') & (df2_no_NaN['Price'] == 3.0)
drinks_p3 = df2_no_NaN[nan_drinks_p3][['Category', 'Item']]['Item'].mode([0])

nan_drinks_p2_5 = (df2_no_NaN['Category'] == 'Drinks') & (df2_no_NaN['Price'] == 2.5)
drinks_p2_5 = df2_no_NaN[nan_drinks_p2_5][['Category', 'Item']]['Item'].mode([0])

# side dishes
nan_side_dishes_p5 = (df2_no_NaN['Category'] == 'Side Dishes') & (df2_no_NaN['Price'] == 5.0)
side_dishes_p5 = df2_no_NaN[nan_side_dishes_p5][['Category', 'Item']]['Item'].mode([0])

nan_side_dishes_p4 = (df2_no_NaN['Category'] == 'Side Dishes') & (df2_no_NaN['Price'] == 4.0)
side_dishes_p4 = df2_no_NaN[nan_side_dishes_p4][['Category', 'Item']]['Item'].mode([0])

# starters
nan_starters_p5 = (df2_no_NaN['Category'] == 'Starters') & (df2_no_NaN['Price'] == 5.0)
starters_p5 = df2_no_NaN[nan_starters_p5][['Category', 'Item']]['Item'].mode([0])


#Examine menu with total count for each item
# columns_filter = ['Category', 'Item', 'Price']
# df2[columns_filter].groupby(['Category', 'Item']).count()
#Purpose: Check existing data that has the most items for certain parameters
#as is total items for duplicate price in each categories

In [ ]:
#Handling Category for the rest and non NaN Price
#Desserts Price 6.0
#PENDING SELALU ERROR DI SINI. PANDAS GK MAU ISI
na_desserts_p6 = df2['Price'] == 6.0
df2_na_desserts_p6 = df2[na_desserts & na_desserts_p6 & na_item]
df2_na_desserts_p6.fillna(desserts_p6, inplace=True)
# df2_na_desserts_p6
# df2_na_desserts_p6['Item'] = df2_na_desserts_p6['Item'].fillna(desserts_p6) #dilarang
# df2_na_desserts_p6

# #Drinks Price 3.0
# na_drinks_p3 = df2['Price'] == 3.0
# df2_na_drinks_p3 = df2[na_drinks & na_drinks_p3 & na_item]
# df2_na_drinks_p3['Price'] = df2_na_drinks_p3['Price'].fillna(drinks_p3)

# #Drinks Price 2.5
# na_drinks_p2_5 = df2['Price'] == 2.5
# df2_na_drinks_p2_5 = df2[na_drinks & na_drinks_p2_5 & na_item]
# df2_na_drinks_p2_5['Price'] = df2_na_drinks_p2_5['Price'].fillna(drinks_p2_5)

# #Side Dishes Price 5.0
# na_side_dishes_p5 = df2['Price'] == 5.0
# df2_na_side_dishes_p5 = df2[na_side_dishes & na_side_dishes_p5 & na_item]
# df2_na_side_dishes_p5['Price'] = df2_na_side_dishes_p5['Price'].fillna(side_dishes_p5)

# #Side Dishes Price 4.0
# na_side_dishes_p4 = df2['Price'] == 4.0
# df2_na_side_dishes_p4 = df2[na_side_dishes & na_side_dishes_p4 & na_item]
# df2_na_side_dishes_p4['Price'] = df2_na_side_dishes_p4['Price'].fillna(side_dishes_p4)

# #Starters 5.0
# na_starters_p5 = df2['Price'] == 5.0
# df2_na_starters_p5 = df2[na_starters & na_starters_p5 & na_item]
# df2_na_starters_p5['Price'] = df2_na_starters_p5['Price'].fillna(starters_p5)

C:\Users\wicak\AppData\Local\Temp\ipykernel_14732\440847867.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_na_desserts_p6.fillna(desserts_p6, inplace=True)


In [ ]:
#OLD SCRIPT
#Replace Na Value with certain value
dessert_6 = 'Chocolate Cake'
drink_3 = 'Orange Juice'
drink_2_5 = 'Coca Cola'
sideD_5 = 'Grilled Vegetables'
sideD_4 = 'Mashed Potatoes'
starters_5 = 'Cheeese Fries'

#Make variable to make more readable
na_dessert = df2['Category']=='Desserts'
na_6 = df2['Price']==6.0

na_drink = df2['Category']=='Drinks'
na_3 = df2['Price']==3.0
na_2_5 = df2['Price']==2.5

na_sideD = df2['Category']=='Side Dishes'
na_5 = df2['Price']==5.0
na_4 = df2['Price']==4.0

na_starters = df2['Category']=='Starters'
#na_5

na_item = df2['Item'].isna()


#fill_value
# df2[na_dessert & na_6 & na_item] = df2[na_dessert & na_6 & na_item].fillna(dessert_6)
# df2[na_drink & na_3 & na_item] = df2[na_drink & na_3 & na_item].fillna(drink_3)
# df2[na_drink & na_2_5 & na_item] = df2[na_drink & na_2_5 & na_item].fillna(drink_2_5)
# df2[na_sideD & na_5 & na_item] = df2[na_sideD & na_5 & na_item].fillna(sideD_5)
# df2[na_sideD & na_4 & na_item] = df2[na_sideD & na_4 & na_item].fillna(sideD_4)
# df2[na_starters & na_5 & na_item] = df2[na_starters & na_5 & na_item].fillna(starters_5)

#Check NaN Values for Cateogry, Item, Price
# na_value = ['Item', 'Price', 'Quantity', 'Order Total']
# df2[df2[na_value].isna().any(axis=1)]

In [34]:
df2[na_dessert & na_6 & na_item]

,Order ID,Customer ID,Category,Item,Price,Quantity,Order Total,Order Date,Payment Method
8,ORD_428611,CUST_083,Desserts,NaN,6.0,2.0,12.0,2023-12-16,Cash
78,ORD_854529,CUST_096,Desserts,NaN,6.0,2.0,12.0,2022-07-08,Credit Card
123,ORD_167242,CUST_050,Desserts,NaN,6.0,3.0,18.0,2023-11-10,Digital Wallet
461,ORD_115022,CUST_082,Desserts,NaN,6.0,1.0,6.0,2022-10-09,Digital Wallet
494,ORD_294858,CUST_036,Desserts,NaN,6.0,3.0,18.0,2022-11-06,Cash
...,...,...,...,...,...,...,...,...,...
15339,ORD_617907,CUST_010,Desserts,NaN,6.0,3.0,18.0,2023-06-12,Cash
15595,ORD_774084,CUST_095,Desserts,NaN,6.0,5.0,30.0,2023-04-04,Credit Card
16075,ORD_110461,CUST_078,Desserts,NaN,6.0,4.0,24.0,2023-10-20,NaN
17062,ORD_935463,CUST_084,Desserts,NaN,6.0,4.0,24.0,2023-12-03,Credit Card


### Numeric Datatype

In [35]:
#To handling null value for numeric datatype, use the relation between price, quantity, and order total
# df[df.isna().any(axis=1)] #show all nan values

#Handling price column
print ("Handling NaN value in Price column")
print ("Before: ", df2['Price'].isna().sum())
df2['Price'] = df2['Price'].fillna(df2['Order Total']/df2['Quantity']) #target fill na 876
print ("After: ", df2['Price'].isna().sum(), "\n") #remaining na is 430. It is suspected that three columns has nan value since two other columns has 430 na records (Quantity and Order Total)

#Handling Quantity Column
print ("Handling NaN value in Quantity column")
print ("Before: ", df2['Quantity'].isna().sum())
df2['Quantity'] = df2['Quantity'].fillna(df2['Order Total']/df2['Price']) #target fill na 430
print ("After: ", df2['Quantity'].isna().sum(), "\n") #still 430.

#Handling order total Column
print ("Handling NaN value in Order Total column")
print ("Before: ", df2['Order Total'].isna().sum())
df2['Order Total'] = df2['Order Total'].fillna(df2['Price']*df2['Quantity']) #target fill na 430
print ("After: ", df2['Order Total'].isna().sum()) #still 430.



Handling NaN value in Price column
Before:  876
After:  430 

Handling NaN value in Quantity column
Before:  430
After:  430 

Handling NaN value in Order Total column
Before:  430
After:  430


# Analyzing Datasets
Do analyzing data by making aggregations, graphics, give short recommendations etc

# Conclusions
Explain conclusions with actionable act and ensure answering KPI